# 🔥 KURE 기반 한국형 번아웃 감정 분류 모델
## AI Hub 데이터셋 활용 버전 (Fixed)

**수정사항**: inference_mode 텐서 에러 해결

**번아웃 카테고리 (박수정 외, 2018)**:
- 0: 정서적_고갈
- 1: 좌절_압박
- 2: 부정적_대인관계
- 3: 자기비하

## 1. 환경 설정

In [ ]:
!pip install -q sentence-transformers transformers torch pandas scikit-learn tqdm openpyxl

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from typing import List, Dict, Optional, Tuple
from tqdm.auto import tqdm
import json
import os
import re
from collections import Counter

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 2. Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 데이터 경로 설정 (본인 경로에 맞게 수정!)
DATA_PATH = "/content/drive/MyDrive/Burnout/dataset"

if os.path.exists(DATA_PATH):
    print(f"✓ 데이터 경로 확인됨: {DATA_PATH}")
    print(f"  폴더 내용: {os.listdir(DATA_PATH)}")
else:
    print(f"❌ 경로를 찾을 수 없음: {DATA_PATH}")

## 3. 번아웃 카테고리 & 감정 매핑

In [ ]:
BURNOUT_CATEGORIES = {
    0: "정서적_고갈",
    1: "좌절_압박",
    2: "부정적_대인관계",
    3: "자기비하"
}

EMOTION_TYPE_TO_BURNOUT = {
    # 정서적 고갈 (0)
    "E20": 0, "E22": 0, "E24": 0, "E26": 0, "E40": 0, "E53": 0, "E32": 0,
    # 좌절/압박 (1)
    "E10": 1, "E15": 1, "E18": 1, "E19": 1, "E21": 1, "E47": 1,
    # 부정적 대인관계 (2)
    "E42": 2, "E49": 2, "E51": 2,
    # 자기비하 (3)
    "E16": 3, "E25": 3, "E30": 3, "E35": 3, "E37": 3, "E50": 3, "E54": 3, "E56": 3, "E57": 3, "E58": 3,
    # 긍정 (제외)
    "E62": -1, "E64": -1, "E66": -1, "E67": -1, "E68": -1, "E69": -1,
}

KEYWORD_TO_BURNOUT = {
    "지침": 0, "피곤": 0, "무기력": 0, "우울": 0, "슬픔": 0, "공허": 0, "외로": 0, "힘들": 0, "지쳐": 0, "스트레스": 0,
    "분노": 1, "화남": 1, "화가": 1, "짜증": 1, "억울": 1, "불만": 1, "실망": 1, "성가": 1, "귀찮": 1,
    "배신": 2, "갈등": 2, "무시": 2, "소외": 2, "서운": 2, "따돌": 2, "어색": 2,
    "불안": 3, "걱정": 3, "초조": 3, "두려": 3, "부끄러": 3, "창피": 3, "자책": 3, "당황": 3, "열등": 3,
}

print("✓ 매핑 테이블 정의 완료")

## 4. 데이터 로더

In [ ]:
def clean_text(text: str) -> str:
    if not text:
        return ""
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    return text


def get_burnout_label(emotion_type: str, text: str = "") -> int:
    if emotion_type in EMOTION_TYPE_TO_BURNOUT:
        return EMOTION_TYPE_TO_BURNOUT[emotion_type]
    for keyword, label in KEYWORD_TO_BURNOUT.items():
        if keyword in text.lower():
            return label
    return -1


def load_emotional_dialogue_json(json_path: str) -> pd.DataFrame:
    print(f"📂 Loading: {json_path}")
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    records = []
    for item in tqdm(data, desc="Parsing"):
        try:
            emotion_info = item.get('profile', {}).get('emotion', {})
            emotion_type = emotion_info.get('type', '')
            content = item.get('talk', {}).get('content', {})

            for key in ['HS01', 'HS02', 'HS03']:
                text = content.get(key, '')
                if text and len(text) >= 10:
                    text = clean_text(text)
                    burnout_label = get_burnout_label(emotion_type, text)
                    records.append({
                        'text': text,
                        'emotion_type': emotion_type,
                        'burnout_label': burnout_label
                    })
        except:
            continue

    df = pd.DataFrame(records)
    print(f"✓ Loaded {len(df)} samples")
    return df


print("✓ 데이터 로더 정의 완료")

## 5. 데이터 로드

In [ ]:
emotional_json_path = os.path.join(
    DATA_PATH,
    "018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.json"
)

if os.path.exists(emotional_json_path):
    df_raw = load_emotional_dialogue_json(emotional_json_path)
else:
    print(f"❌ 파일 없음: {emotional_json_path}")
    df_raw = pd.DataFrame()

In [ ]:
# 필터링
if len(df_raw) > 0:
    df_filtered = df_raw[df_raw['burnout_label'] != -1].copy()
    df_filtered = df_filtered.drop_duplicates(subset=['text'])

    print(f"\n📊 데이터: {len(df_filtered)}개")
    print("\n📈 카테고리 분포:")
    for label, name in BURNOUT_CATEGORIES.items():
        count = len(df_filtered[df_filtered['burnout_label'] == label])
        print(f"   {name}: {count}개 ({100*count/len(df_filtered):.1f}%)")
else:
    df_filtered = pd.DataFrame()

## 6. Train/Val 분할

In [ ]:
from sklearn.model_selection import train_test_split

if len(df_filtered) > 100:
    train_df, val_df = train_test_split(
        df_filtered, test_size=0.1,
        stratify=df_filtered['burnout_label'], random_state=42
    )
    print(f"Train: {len(train_df)} / Val: {len(val_df)}")
else:
    train_df, val_df = df_filtered, None
    print("데이터 부족, val 없이 진행")

## 7. 🔑 임베딩 미리 계산 (핵심 수정!)

**문제**: `SentenceTransformer.encode()`가 `inference_mode`로 실행되어 backward와 충돌

**해결**: 임베딩을 미리 계산해서 저장 → 학습 시에는 임베딩만 사용

In [ ]:
# KURE 모델 로드
print("📥 Loading KURE model...")
kure_model = SentenceTransformer("nlpai-lab/KURE-v1", device=device)
EMBEDDING_DIM = kure_model.get_sentence_embedding_dimension()
print(f"✓ KURE loaded, embedding dim: {EMBEDDING_DIM}")

In [ ]:
# 임베딩 미리 계산 (학습 전에 한 번만!)
print("\n🔄 Pre-computing embeddings...")

train_texts = train_df['text'].tolist()
train_labels = train_df['burnout_label'].tolist()

# 배치로 임베딩 계산
with torch.no_grad():
    train_embeddings = kure_model.encode(
        train_texts,
        batch_size=64,
        show_progress_bar=True,
        convert_to_tensor=True
    )

print(f"✓ Train embeddings: {train_embeddings.shape}")

# Validation
if val_df is not None and len(val_df) > 0:
    val_texts = val_df['text'].tolist()
    val_labels = val_df['burnout_label'].tolist()

    with torch.no_grad():
        val_embeddings = kure_model.encode(
            val_texts,
            batch_size=64,
            show_progress_bar=True,
            convert_to_tensor=True
        )
    print(f"✓ Val embeddings: {val_embeddings.shape}")
else:
    val_embeddings, val_labels = None, None

## 8. 분류 모델 정의 (임베딩 기반)

In [ ]:
class BurnoutClassifier(nn.Module):
    """
    분류 헤드만 학습 (임베딩은 미리 계산됨)
    """
    def __init__(self, input_dim=1024, hidden_dim=256, num_classes=4, dropout=0.3):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, embeddings):
        return self.classifier(embeddings)


class EmbeddingDataset(Dataset):
    """미리 계산된 임베딩 사용"""
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]


print("✓ 모델 클래스 정의 완료")

## 9. 학습

In [ ]:
def train_classifier(model, train_embeddings, train_labels,
                     val_embeddings=None, val_labels=None,
                     epochs=10, batch_size=64, lr=1e-3, device='cuda'):

    train_dataset = EmbeddingDataset(train_embeddings, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_loader = None
    if val_embeddings is not None:
        val_dataset = EmbeddingDataset(val_embeddings, val_labels)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    history = {'train_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for emb, labels in pbar:
            emb, labels = emb.to(device), labels.to(device)

            optimizer.zero_grad()
            logits = model(emb)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, pred = torch.max(logits, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        train_acc = 100 * correct / total
        history['train_loss'].append(total_loss / len(train_loader))
        history['train_acc'].append(train_acc)

        # Validation
        val_acc = 0
        if val_loader:
            model.eval()
            val_correct, val_total = 0, 0
            with torch.no_grad():
                for emb, labels in val_loader:
                    emb, labels = emb.to(device), labels.to(device)
                    logits = model(emb)
                    _, pred = torch.max(logits, 1)
                    val_total += labels.size(0)
                    val_correct += (pred == labels).sum().item()
            val_acc = 100 * val_correct / val_total
            history['val_acc'].append(val_acc)

        print(f"  Loss: {history['train_loss'][-1]:.4f} | Train: {train_acc:.1f}% | Val: {val_acc:.1f}%")

    return model, history


print("✓ Training 함수 정의 완료")

In [ ]:
# 모델 생성 & 학습
classifier = BurnoutClassifier(
    input_dim=EMBEDDING_DIM,
    hidden_dim=256,
    num_classes=4,
    dropout=0.3
).to(device)

print(f"\n🚀 학습 시작!")
print(f"   Train: {len(train_labels)}개")
print(f"   Val: {len(val_labels) if val_labels else 0}개")

classifier, history = train_classifier(
    model=classifier,
    train_embeddings=train_embeddings,
    train_labels=train_labels,
    val_embeddings=val_embeddings,
    val_labels=val_labels,
    epochs=15,
    batch_size=64,
    lr=1e-3,
    device=device
)

## 10. 모델 저장

In [ ]:
# 저장
save_path = "/content/drive/MyDrive/Burnout/burnout_classifier.pt"
torch.save({
    'model_state_dict': classifier.state_dict(),
    'embedding_dim': EMBEDDING_DIM,
    'num_classes': 4
}, save_path)
print(f"✓ Saved: {save_path}")

## 11. 추론 함수

In [ ]:
def predict(texts, kure_model, classifier, device='cuda'):
    """
    텍스트 → 번아웃 카테고리 예측
    """
    classifier.eval()

    # 1. KURE로 임베딩
    with torch.no_grad():
        embeddings = kure_model.encode(
            texts,
            convert_to_tensor=True,
            show_progress_bar=False
        ).to(device)

        # 2. 분류
        logits = classifier(embeddings)
        probs = F.softmax(logits, dim=-1)
        labels = torch.argmax(logits, dim=-1).cpu().tolist()

    categories = [BURNOUT_CATEGORIES[l] for l in labels]
    return labels, categories, probs


print("✓ 추론 함수 정의 완료")

## 12. 테스트

In [ ]:
test_texts = [
    "오늘도 야근이다. 너무 지치고 힘들다.",
    "상사가 또 화를 냈다. 억울하고 분하다.",
    "팀원들이 나를 무시하는 것 같다.",
    "나는 왜 이것밖에 못하는 걸까. 불안하다.",
    "일은 왜 해도 해도 끝이 없을까? 화가 난다.",
]

labels, categories, probs = predict(test_texts, kure_model, classifier, device)

print("\n" + "="*60)
print("🔮 예측 결과")
print("="*60)
for text, cat, prob in zip(test_texts, categories, probs):
    print(f"\n입력: {text}")
    print(f"예측: {cat}")
    prob_str = {k: f"{v:.1%}" for k, v in zip(BURNOUT_CATEGORIES.values(), prob.cpu().numpy())}
    print(f"확률: {prob_str}")

## 13. 커스텀 테스트

In [ ]:
my_text = "월요일 아침부터 야근 예고를 받았다. 벌써 지친다."  # @param {type:"string"}

labels, categories, probs = predict([my_text], kure_model, classifier, device)

print(f"\n입력: {my_text}")
print(f"\n🏷️ 예측: {categories[0]}")
print(f"\n📊 확률:")
for name, p in zip(BURNOUT_CATEGORIES.values(), probs[0].cpu().numpy()):
    bar = "█" * int(p * 20)
    print(f"  {name}: {bar} {p:.1%}")

## 14. 학습 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()

# Accuracy
axes[1].plot(history['train_acc'], label='Train Acc')
if history['val_acc']:
    axes[1].plot(history['val_acc'], label='Val Acc')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n최종 성능:")
print(f"  Train Acc: {history['train_acc'][-1]:.1f}%")
if history['val_acc']:
    print(f"  Val Acc: {history['val_acc'][-1]:.1f}%")